## Federated Learning with one corrupted node: geometric median
Using one balanced and one unbalanced dataset with one corrupted node (5%, 25% and 50% corrupted samples) to test different aggregation functions and determine the more robust one. 

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
def weiszfeld_update(points, tol=1e-5, max_iter=100):
    """
    Compute the geometric median using Weiszfeld's algorithm.

    Args:
    points: A list of points (numpy arrays) to find the geometric median of.
    tol: Tolerance for stopping criterion.
    max_iter: Maximum number of iterations.

    Returns:
    The geometric median of the points.
    """
    points = np.array(points)
    median = np.mean(points, axis=0)

    for _ in range(max_iter):
        distances = np.linalg.norm(points - median, axis=1)
        nonzero_distances = np.where(distances != 0, distances, np.finfo(float).eps)
        weights = 1 / nonzero_distances
        new_median = np.sum(points * weights[:, None], axis=0) / np.sum(weights)

        if np.linalg.norm(new_median - median) < tol:
            return new_median

        median = new_median

    return median

def aggregate(grad_list):
    """
    Apply Geometric Median aggregation method to a list of gradients.

    Args:
    grad_list: List of gradients from different models. Each element in the list
               is a list of gradients for each layer of a model.

    Returns:
    The geometric median of the gradients.
    """
    # Flatten gradients to compute geometric median
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute geometric median using Weiszfeld's algorithm
    flat_median = weiszfeld_update(flat_grads)

    # Reshape the flat median back to the original shape
    median_grad = []
    index = 0
    for grad in grad_list[0]:
        shape = tf.shape(grad)
        size = tf.reduce_prod(shape)
        median_grad.append(tf.reshape(flat_median[index:index + size], shape))
        index += size

    return median_grad

In [6]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3836997398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [7]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcomplete, ycomplete = preprocessing(test_complete)

#### 5A 5% 

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3687049449.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3687049449.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3687049449.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3687049449.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on im

In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Ageo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Ageo.hdf5')

Epoch 1/5
31/31 [==============================] - 13s 418ms/step - loss: 0.4778 - accuracy: 0.8250 - val_loss: 0.2640 - val_accuracy: 0.8810
Epoch 2/5
31/31 [==============================] - 11s 346ms/step - loss: 0.1970 - accuracy: 0.9489 - val_loss: 0.1787 - val_accuracy: 0.9613
Epoch 3/5
31/31 [==============================] - 10s 323ms/step - loss: 0.1663 - accuracy: 0.9648 - val_loss: 0.1628 - val_accuracy: 0.9633
Epoch 4/5
31/31 [==============================] - 9s 297ms/step - loss: 0.1539 - accuracy: 0.9662 - val_loss: 0.1542 - val_accuracy: 0.9649
Epoch 5/5
31/31 [==============================] - 12s 384ms/step - loss: 0.1465 - accuracy: 0.9675 - val_loss: 0.1489 - val_accuracy: 0.9655
Epoch 1/5
31/31 [==============================] - 14s 442ms/step - loss: 0.0338 - accuracy: 0.9898 - val_loss: 0.3732 - val_accuracy: 0.7678
Epoch 2/5
31/31 [==============================] - 10s 324ms/step - loss: 0.0235 - accuracy: 0.9909 - val_loss: 0.4554 - val_accuracy: 0.6506
Epoch 3

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.04889281094074249, 0.4478214681148529], [0.05990856885910034, 0.5273932814598083], [0.07000613957643509, 0.5446551442146301]]
Accuracy for iterations:  [[0.9735750555992126, 0.7613165974617004], [0.9738233685493469, 0.7479655742645264], [0.9727862477302551, 0.7575383186340332]]
F1 for iterations:  [[0.9735385254940947, 0.7398564684871461], [0.9737850415692508, 0.7234486441670435], [0.9727401086043492, 0.7360250499662067]]
Precision for iterations:  [[0.9741483887157147, 0.8220007622883527], [0.9744563735051996, 0.8115405582179294], [0.9735887599953557, 0.8154317890204095]]
Recall for iterations:  [[0.973575038709866, 0.7613165734214924], [0.973823366151509, 0.7479655598343384], [0.972786233895235, 0.7575383273995495]]


#### 5B 10%

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/2463029928.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/2463029928.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/2463029928.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/2463029928.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Ageo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Ageo.hdf5')

Epoch 1/5
31/31 [==============================] - 13s 420ms/step - loss: 0.4867 - accuracy: 0.8029 - val_loss: 0.2931 - val_accuracy: 0.9085
Epoch 2/5
31/31 [==============================] - 10s 308ms/step - loss: 0.2632 - accuracy: 0.9302 - val_loss: 0.2451 - val_accuracy: 0.9389
Epoch 3/5
31/31 [==============================] - 9s 279ms/step - loss: 0.2410 - accuracy: 0.9404 - val_loss: 0.2314 - val_accuracy: 0.9418
Epoch 4/5
31/31 [==============================] - 12s 393ms/step - loss: 0.2301 - accuracy: 0.9420 - val_loss: 0.2244 - val_accuracy: 0.9426
Epoch 5/5
31/31 [==============================] - 13s 425ms/step - loss: 0.2242 - accuracy: 0.9426 - val_loss: 0.2206 - val_accuracy: 0.9426
Epoch 1/5
31/31 [==============================] - 13s 422ms/step - loss: 0.0360 - accuracy: 0.9896 - val_loss: 0.4871 - val_accuracy: 0.6711
Epoch 2/5
31/31 [==============================] - 10s 315ms/step - loss: 0.0232 - accuracy: 0.9910 - val_loss: 0.4265 - val_accuracy: 0.6665
Epoch 3

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.05741056054830551, 0.4590655267238617], [0.07663948088884354, 0.45165905356407166], [0.06982630491256714, 0.3087761104106903]]
Accuracy for iterations:  [[0.970149576663971, 0.7760117650032043], [0.9688495397567749, 0.770943820476532], [0.971741795539856, 0.8325583338737488]]
F1 for iterations:  [[0.9700949410282225, 0.7582688345798568], [0.9687809597016985, 0.751831152230553], [0.971691480828758, 0.8248670942131826]]
Precision for iterations:  [[0.9710471170781172, 0.8298399379845439], [0.9700489101010882, 0.8278210977491526], [0.9726070256090559, 0.8635768653781645]]
Recall for iterations:  [[0.9701495807648485, 0.7760117706895299], [0.9688495135703643, 0.7709438349197123], [0.9717417978906775, 0.8325583085083194]]


#### 5B 25%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3855986297.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3855986297.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3855986297.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2424/3855986297.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Ageo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Ageo.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 264ms/step - loss: 0.4617 - accuracy: 0.8634 - val_loss: 0.2182 - val_accuracy: 0.9276
Epoch 2/5
31/31 [==============================] - 9s 301ms/step - loss: 0.1508 - accuracy: 0.9629 - val_loss: 0.1238 - val_accuracy: 0.9755
Epoch 3/5
31/31 [==============================] - 12s 382ms/step - loss: 0.1166 - accuracy: 0.9771 - val_loss: 0.1093 - val_accuracy: 0.9776
Epoch 4/5
31/31 [==============================] - 13s 418ms/step - loss: 0.1038 - accuracy: 0.9791 - val_loss: 0.1026 - val_accuracy: 0.9787
Epoch 5/5
31/31 [==============================] - 13s 417ms/step - loss: 0.0988 - accuracy: 0.9798 - val_loss: 0.1013 - val_accuracy: 0.9787
Epoch 1/5
31/31 [==============================] - 10s 337ms/step - loss: 0.0336 - accuracy: 0.9899 - val_loss: 0.4402 - val_accuracy: 0.7092
Epoch 2/5
31/31 [==============================] - 9s 305ms/step - loss: 0.0230 - accuracy: 0.9907 - val_loss: 0.3895 - val_accuracy: 0.6843
Epoch 3/5

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.045988935977220535, 0.30304765701293945], [0.05962508171796799, 0.47183671593666077], [0.052721574902534485, 0.33591940999031067]]
Accuracy for iterations:  [[0.9753279089927673, 0.8548281788825989], [0.9726255536079407, 0.7662755250930786], [0.975846529006958, 0.8447467684745789]]
F1 for iterations:  [[0.9752996501961515, 0.8497638865929611], [0.9725824306937283, 0.7458750322544874], [0.9758194239809572, 0.8385634886002252]]
Precision for iterations:  [[0.9757488527705024, 0.8775109729448786], [0.9733359605589635, 0.825845210382415], [0.9762527430319973, 0.8712136934730742]]
Recall for iterations:  [[0.9753279382979345, 0.8548281624645789], [0.9726255514329954, 0.7662755213252924], [0.9758465044260715, 0.8447467848579525]]


#### 5B 5% 

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_18408/3503541895.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_18408/3503541895.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_18408/3503541895.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')


In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Bgeo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bgeo.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 258ms/step - loss: 0.4721 - accuracy: 0.8053 - val_loss: 0.2419 - val_accuracy: 0.9303
Epoch 2/5
31/31 [==============================] - 8s 261ms/step - loss: 0.1975 - accuracy: 0.9519 - val_loss: 0.1725 - val_accuracy: 0.9627
Epoch 3/5
31/31 [==============================] - 8s 250ms/step - loss: 0.1680 - accuracy: 0.9649 - val_loss: 0.1596 - val_accuracy: 0.9647
Epoch 4/5
31/31 [==============================] - 14s 452ms/step - loss: 0.1544 - accuracy: 0.9665 - val_loss: 0.1509 - val_accuracy: 0.9660
Epoch 5/5
31/31 [==============================] - 23s 734ms/step - loss: 0.1469 - accuracy: 0.9675 - val_loss: 0.1460 - val_accuracy: 0.9660
Epoch 1/5
31/31 [==============================] - 7s 236ms/step - loss: 0.0317 - accuracy: 0.9905 - val_loss: 0.2953 - val_accuracy: 0.8311
Epoch 2/5
31/31 [==============================] - 18s 583ms/step - loss: 0.0230 - accuracy: 0.9912 - val_loss: 0.3854 - val_accuracy: 0.6841
Epoch 3/5


In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.11491592228412628, 0.24709856510162354], [0.10359504073858261, 0.30141469836235046], [0.08666576445102692, 0.365721732378006]]
Accuracy for iterations:  [[0.9255090951919556, 0.8544648885726929], [0.9423003792762756, 0.8314865827560425], [0.9570101499557495, 0.8111603856086731]]
F1 for iterations:  [[0.9248198198004561, 0.8493405193344246], [0.9419529878843841, 0.8237084306097469], [0.9568479638750795, 0.8002756404314799]]
Precision for iterations:  [[0.9337741252821732, 0.8774352711870831], [0.9471440452961941, 0.862565997012988], [0.9596152895107926, 0.8506515144349549]]
Recall for iterations:  [[0.9255090712553683, 0.8544648695778536], [0.9423003885594087, 0.8314865944924799], [0.9570101376026177, 0.8111603574802005]]


#### 5B 10%

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Bgeo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Bgeo.hdf5')

Epoch 1/5
31/31 [==============================] - 4s 138ms/step - loss: 0.4904 - accuracy: 0.8183 - val_loss: 0.2862 - val_accuracy: 0.9218
Epoch 2/5
31/31 [==============================] - 4s 141ms/step - loss: 0.2706 - accuracy: 0.9273 - val_loss: 0.2487 - val_accuracy: 0.9396
Epoch 3/5
31/31 [==============================] - 4s 138ms/step - loss: 0.2459 - accuracy: 0.9391 - val_loss: 0.2341 - val_accuracy: 0.9417
Epoch 4/5
31/31 [==============================] - 4s 135ms/step - loss: 0.2323 - accuracy: 0.9412 - val_loss: 0.2234 - val_accuracy: 0.9429
Epoch 5/5
31/31 [==============================] - 4s 137ms/step - loss: 0.2236 - accuracy: 0.9424 - val_loss: 0.2185 - val_accuracy: 0.9433
Epoch 1/5
31/31 [==============================] - 4s 141ms/step - loss: 0.0379 - accuracy: 0.9899 - val_loss: 0.3268 - val_accuracy: 0.8140
Epoch 2/5
31/31 [==============================] - 4s 143ms/step - loss: 0.0239 - accuracy: 0.9909 - val_loss: 0.3649 - val_accuracy: 0.7392
Epoch 3/5
31/

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.11470375955104828, 0.23222364485263824], [0.09963300824165344, 0.259491503238678], [0.09414619952440262, 0.26222124695777893]]
Accuracy for iterations:  [[0.9249832034111023, 0.8733197450637817], [0.9408031105995178, 0.8527936935424805], [0.9483625292778015, 0.8646552562713623]]
F1 for iterations:  [[0.9242816503937178, 0.8698344379594948], [0.940434626040827, 0.8475370303696735], [0.948103720135536, 0.8604375955006761]]
Precision for iterations:  [[0.9333531466382072, 0.8905279448236121], [0.9458517835799812, 0.8760728745656486], [0.9521770208609084, 0.8846596668569583]]
Recall for iterations:  [[0.9249832013789476, 0.8733197703988956], [0.9408031201612668, 0.8527937222989174], [0.9483624996348126, 0.8646552350504977]]


#### 5B 25%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Bgeo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Bgeo.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 145ms/step - loss: 0.5577 - accuracy: 0.7447 - val_loss: 0.4328 - val_accuracy: 0.8525
Epoch 2/5
31/31 [==============================] - 4s 141ms/step - loss: 0.4210 - accuracy: 0.8598 - val_loss: 0.4024 - val_accuracy: 0.8689
Epoch 3/5
31/31 [==============================] - 5s 149ms/step - loss: 0.4061 - accuracy: 0.8665 - val_loss: 0.3930 - val_accuracy: 0.8697
Epoch 4/5
31/31 [==============================] - 6s 192ms/step - loss: 0.3973 - accuracy: 0.8676 - val_loss: 0.3893 - val_accuracy: 0.8705
Epoch 5/5
31/31 [==============================] - 6s 194ms/step - loss: 0.3939 - accuracy: 0.8684 - val_loss: 0.3869 - val_accuracy: 0.8705
Epoch 1/5
31/31 [==============================] - 7s 232ms/step - loss: 0.0492 - accuracy: 0.9897 - val_loss: 0.5851 - val_accuracy: 0.6476
Epoch 2/5
31/31 [==============================] - 11s 345ms/step - loss: 0.0242 - accuracy: 0.9911 - val_loss: 0.4860 - val_accuracy: 0.6481
Epoch 3/5
31

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.13327844440937042, 0.2549484968185425], [0.1026846170425415, 0.2271340936422348], [0.07630041986703873, 0.249038428068161]]
Accuracy for iterations:  [[0.923982560634613, 0.8539562821388245], [0.9342954754829407, 0.8732107877731323], [0.9566011428833008, 0.8710673451423645]]
F1 for iterations:  [[0.9232559208613232, 0.8489066584180305], [0.9338084600112092, 0.8697433202461109], [0.9564431931758209, 0.8673570867825495]]
Precision for iterations:  [[0.932574827909274, 0.876227258613864], [0.9405867705057854, 0.8902485428821898], [0.9590661290639726, 0.8893179846143305]]
Recall for iterations:  [[0.9239825878640918, 0.8539562595364383], [0.9342954804405621, 0.873210782532878], [0.956601127698735, 0.8710673545011989]]


#### 5C NODE 1 5%

In [9]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_17588/591709827.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')


In [10]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [11]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C1geo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [12]:
global_model = build_model((24,1,1))

In [13]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [15]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1geo.hdf5')

Epoch 1/5
16/16 [==============================] - 6s 351ms/step - loss: 0.6101 - accuracy: 0.7566 - val_loss: 0.4808 - val_accuracy: 0.8062
Epoch 2/5
16/16 [==============================] - 4s 264ms/step - loss: 0.3725 - accuracy: 0.8689 - val_loss: 0.2576 - val_accuracy: 0.9247
Epoch 3/5
16/16 [==============================] - 4s 239ms/step - loss: 0.2289 - accuracy: 0.9376 - val_loss: 0.2027 - val_accuracy: 0.9511
Epoch 4/5
16/16 [==============================] - 4s 236ms/step - loss: 0.1983 - accuracy: 0.9560 - val_loss: 0.1832 - val_accuracy: 0.9609
Epoch 5/5
16/16 [==============================] - 4s 233ms/step - loss: 0.1840 - accuracy: 0.9616 - val_loss: 0.1710 - val_accuracy: 0.9645
Epoch 1/5
14/14 [==============================] - 4s 262ms/step - loss: 0.0414 - accuracy: 0.9910 - val_loss: 0.6484 - val_accuracy: 0.7275
Epoch 2/5
14/14 [==============================] - 3s 233ms/step - loss: 0.0234 - accuracy: 0.9940 - val_loss: 0.8319 - val_accuracy: 0.7270
Epoch 3/5
14/

In [16]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06918075680732727, 0.1445847898721695], [0.06189906597137451, 0.1246391162276268], [0.0868644118309021, 0.22009800374507904]]
Accuracy for iterations:  [[0.9851734042167664, 0.9568226337432861], [0.988153338432312, 0.9681937098503113], [0.9796006083488464, 0.9219101667404175]]
F1 for iterations:  [[0.9851756418054838, 0.9567013051454595], [0.9881597348862576, 0.9682852036840406], [0.9796161107324796, 0.9221756408354781]]
Precision for iterations:  [[0.9851929503093813, 0.9573274946641492], [0.988441769814709, 0.9703413561437108], [0.9804435791205228, 0.9336873622329488]]
Recall for iterations:  [[0.9851733909842532, 0.9568226404127007], [0.9881533202839697, 0.9681937077672019], [0.9796006310438518, 0.9219101939984015]]


#### 5C NODE 1 10%

In [40]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/372464333.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')


In [41]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [42]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C1geo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [43]:
global_model = build_model((24,1,1))

In [44]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [45]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [46]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1geo.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 237ms/step - loss: 0.6154 - accuracy: 0.7681 - val_loss: 0.4970 - val_accuracy: 0.7811
Epoch 2/5
16/16 [==============================] - 3s 216ms/step - loss: 0.4002 - accuracy: 0.8472 - val_loss: 0.3168 - val_accuracy: 0.9028
Epoch 3/5
16/16 [==============================] - 3s 218ms/step - loss: 0.2936 - accuracy: 0.9172 - val_loss: 0.2823 - val_accuracy: 0.9271
Epoch 4/5
16/16 [==============================] - 3s 203ms/step - loss: 0.2698 - accuracy: 0.9332 - val_loss: 0.2664 - val_accuracy: 0.9379
Epoch 5/5
16/16 [==============================] - 4s 220ms/step - loss: 0.2558 - accuracy: 0.9373 - val_loss: 0.2525 - val_accuracy: 0.9390
Epoch 1/5
14/14 [==============================] - 3s 239ms/step - loss: 0.0486 - accuracy: 0.9908 - val_loss: 0.7414 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 3s 206ms/step - loss: 0.0244 - accuracy: 0.9941 - val_loss: 0.8693 - val_accuracy: 0.7270
Epoch 3/5
14/

In [47]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06640256196260452, 0.1257031261920929], [0.0543942004442215, 0.10962170362472534], [0.051678989082574844, 0.08576647192239761]]
Accuracy for iterations:  [[0.988379716873169, 0.9754777550697327], [0.9897966384887695, 0.976113498210907], [0.9899938702583313, 0.9845600724220276]]
F1 for iterations:  [[0.9883839596066323, 0.975493552587857], [0.9898015906657185, 0.9761696717141854], [0.9899986226483048, 0.9845855778790374]]
Precision for iterations:  [[0.9884906782301287, 0.9755687720040811], [0.9900111184654423, 0.9773419606883076], [0.9902001605974614, 0.9850820400334382]]
Recall for iterations:  [[0.9883797364807619, 0.9754777301460438], [0.9897966636477841, 0.976113492697813], [0.9899938648514418, 0.9845600523141756]]


#### 5C NODE 1 25%

In [6]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/2868219575.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')


In [16]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [17]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C1geo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [18]:
global_model = build_model((24,1,1))

In [19]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [21]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1geo.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 184ms/step - loss: 0.6350 - accuracy: 0.7102 - val_loss: 0.5472 - val_accuracy: 0.7394
Epoch 2/5
16/16 [==============================] - 3s 207ms/step - loss: 0.4898 - accuracy: 0.8046 - val_loss: 0.4409 - val_accuracy: 0.8484
Epoch 3/5
16/16 [==============================] - 3s 185ms/step - loss: 0.4400 - accuracy: 0.8509 - val_loss: 0.4209 - val_accuracy: 0.8651
Epoch 4/5
16/16 [==============================] - 3s 197ms/step - loss: 0.4258 - accuracy: 0.8624 - val_loss: 0.4109 - val_accuracy: 0.8677
Epoch 5/5
16/16 [==============================] - 3s 199ms/step - loss: 0.4166 - accuracy: 0.8637 - val_loss: 0.4042 - val_accuracy: 0.8683
Epoch 1/5
14/14 [==============================] - 3s 180ms/step - loss: 0.0816 - accuracy: 0.9906 - val_loss: 0.5417 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 177ms/step - loss: 0.0281 - accuracy: 0.9937 - val_loss: 0.7455 - val_accuracy: 0.7270
Epoch 3/5
14/

In [22]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07301006466150284, 0.13686226308345795], [0.07973063737154007, 0.1810533106327057], [0.11098721623420715, 0.2670384347438812]]
Accuracy for iterations:  [[0.9883286356925964, 0.9682118892669678], [0.9850346446037292, 0.9388033151626587], [0.9744295477867126, 0.9005667567253113]]
F1 for iterations:  [[0.9883327370084766, 0.9681764578223622], [0.9850441636705817, 0.9390300567638474], [0.9744505128900312, 0.9007852290323695]]
Precision for iterations:  [[0.9884330080269008, 0.9682811247686117], [0.9854926668972157, 0.946286857250159], [0.9757355470526499, 0.9189210677814361]]
Recall for iterations:  [[0.9883286102427766, 0.9682118724115382], [0.9850346197668643, 0.938803313231127], [0.9744295772590493, 0.9005667369032915]]


#### 5C NODE 3 5%

In [23]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/927562370.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')


In [24]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [25]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C3geo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [26]:
global_model = build_model((24,1,1))

In [27]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [28]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [29]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3geo.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 168ms/step - loss: 0.5297 - accuracy: 0.7412 - val_loss: 1.7559 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 170ms/step - loss: 0.1924 - accuracy: 0.9607 - val_loss: 3.6784 - val_accuracy: 0.1332
Epoch 3/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0802 - accuracy: 0.9893 - val_loss: 3.6751 - val_accuracy: 0.1342
Epoch 4/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0515 - accuracy: 0.9893 - val_loss: 1.8339 - val_accuracy: 0.1357
Epoch 5/5
16/16 [==============================] - 3s 202ms/step - loss: 0.0374 - accuracy: 0.9899 - val_loss: 1.5210 - val_accuracy: 0.1622
Epoch 1/5
14/14 [==============================] - 3s 230ms/step - loss: 0.0231 - accuracy: 0.9949 - val_loss: 0.8074 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 197ms/step - loss: 0.0208 - accuracy: 0.9949 - val_loss: 0.5096 - val_accuracy: 0.7351
Epoch 3/5

In [30]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1084146797657013, 0.1123688668012619], [0.1162383109331131, 0.1426282525062561], [0.11805704981088638, 0.16804981231689453]]
Accuracy for iterations:  [[0.9879999160766602, 0.9727712273597717], [0.9890370965003967, 0.952953577041626], [0.9900742173194885, 0.9472498893737793]]
F1 for iterations:  [[0.988003964019539, 0.9727450833085427], [0.9890408833630392, 0.9527551368789061], [0.9900774749113563, 0.9469688561753138]]
Precision for iterations:  [[0.9880929627462135, 0.9728250915394179], [0.9891371143801229, 0.9540567725774562], [0.9901625178749895, 0.9488898335585798]]
Recall for iterations:  [[0.9879999415700137, 0.9727711981399404], [0.9890370738262877, 0.9529535711690765], [0.9900742060825616, 0.9472498728474896]]


#### 5C NODE 3 10%

In [31]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/4114672810.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')


In [32]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [33]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C3geo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [34]:
global_model = build_model((24,1,1))

In [35]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [36]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [37]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3geo.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 212ms/step - loss: 0.5324 - accuracy: 0.7423 - val_loss: 1.7323 - val_accuracy: 0.0233
Epoch 2/5
16/16 [==============================] - 3s 182ms/step - loss: 0.1918 - accuracy: 0.9744 - val_loss: 3.9565 - val_accuracy: 0.1281
Epoch 3/5
16/16 [==============================] - 3s 176ms/step - loss: 0.0867 - accuracy: 0.9892 - val_loss: 3.8688 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0524 - accuracy: 0.9892 - val_loss: 1.6523 - val_accuracy: 0.1386
Epoch 5/5
16/16 [==============================] - 3s 163ms/step - loss: 0.0364 - accuracy: 0.9891 - val_loss: 1.2198 - val_accuracy: 0.2383
Epoch 1/5
14/14 [==============================] - 3s 184ms/step - loss: 0.0220 - accuracy: 0.9948 - val_loss: 0.8367 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 171ms/step - loss: 0.0195 - accuracy: 0.9950 - val_loss: 0.6023 - val_accuracy: 0.7283
Epoch 3/5
14/

In [38]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07911981642246246, 0.10972201079130173], [0.11108022183179855, 0.12621992826461792], [0.10594591498374939, 0.14654435217380524]]
Accuracy for iterations:  [[0.9884600639343262, 0.9746421575546265], [0.989482581615448, 0.9660139679908752], [0.9902129769325256, 0.9479401111602783]]
F1 for iterations:  [[0.9884641874354215, 0.9746253145543056], [0.9894866514296236, 0.9659476699492876], [0.9902162674857102, 0.9476694014296102]]
Precision for iterations:  [[0.9885660727407686, 0.9746612204195951], [0.9896081802206068, 0.9662745375191877], [0.9903052955349512, 0.9495162369660378]]
Recall for iterations:  [[0.9884600777118817, 0.9746421565065756], [0.9894826024715884, 0.9660139504468502], [0.9902129772999503, 0.9479401293322677]]


#### 5C NODE 3 25%

In [39]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/512859027.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')


In [40]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [41]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C3geo.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [42]:
global_model = build_model((24,1,1))

In [43]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [44]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [45]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3geo.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 181ms/step - loss: 0.5393 - accuracy: 0.7489 - val_loss: 1.6758 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 186ms/step - loss: 0.2027 - accuracy: 0.9544 - val_loss: 3.6085 - val_accuracy: 0.1320
Epoch 3/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0847 - accuracy: 0.9892 - val_loss: 4.0488 - val_accuracy: 0.1337
Epoch 4/5
16/16 [==============================] - 3s 204ms/step - loss: 0.0519 - accuracy: 0.9894 - val_loss: 2.0923 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 206ms/step - loss: 0.0370 - accuracy: 0.9903 - val_loss: 1.6371 - val_accuracy: 0.1401
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0219 - accuracy: 0.9950 - val_loss: 0.8711 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 219ms/step - loss: 0.0203 - accuracy: 0.9950 - val_loss: 0.6223 - val_accuracy: 0.7294
Epoch 3/5

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3geo.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 181ms/step - loss: 0.5393 - accuracy: 0.7489 - val_loss: 1.6758 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 186ms/step - loss: 0.2027 - accuracy: 0.9544 - val_loss: 3.6085 - val_accuracy: 0.1320
Epoch 3/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0847 - accuracy: 0.9892 - val_loss: 4.0488 - val_accuracy: 0.1337
Epoch 4/5
16/16 [==============================] - 3s 204ms/step - loss: 0.0519 - accuracy: 0.9894 - val_loss: 2.0923 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 206ms/step - loss: 0.0370 - accuracy: 0.9903 - val_loss: 1.6371 - val_accuracy: 0.1401
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0219 - accuracy: 0.9950 - val_loss: 0.8711 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 219ms/step - loss: 0.0203 - accuracy: 0.9950 - val_loss: 0.6223 - val_accuracy: 0.7294
Epoch 3/5

In [46]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08544706553220749, 0.12328588217496872], [0.09680859744548798, 0.13693860173225403], [0.12291232496500015, 0.15408408641815186]]
Accuracy for iterations:  [[0.9878538846969604, 0.9722262620925903], [0.9889128804206848, 0.9629804491996765], [0.9890589714050293, 0.9475768208503723]]
F1 for iterations:  [[0.9878578666501733, 0.9722010170737153], [0.988916709205785, 0.9628907034001225], [0.9890627905095888, 0.9472986746263102]]
Precision for iterations:  [[0.9879434565329552, 0.9722715522619876], [0.9890099785924518, 0.9633804123844554], [0.9891592171717783, 0.949208839513966]]
Recall for iterations:  [[0.9878538666043414, 0.9722262588098525], [0.9889129101054661, 0.9629804548426942], [0.9890589850711385, 0.9475768364455424]]
